In [68]:
import logging
import pathlib

import hydra

from pathlib import Path

from dataset.monthlysampler import loader
from dataset.augmentations import build_transform

ROOT_PATH = "/data/Prosjekter3/154012_monitoring_natural_habitat_loss_in_norway_with_cop/R/DATA/For_MSc/Project_1/"


root = Path(ROOT_PATH)
SIZE=256
LENGTH=30
BATCH_SIZE_FOR_STAT=16
BATCH_SIZE_FOR_TRAINING=8

train_path_imgs = root/'tra_scene'
train_path_masks = root/'tra_truth'
val_path_imgs = root/'val_scene'
val_path_masks = root/'val_truth'


In [96]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
# Compute band mins and band maxes for image normalisation
stat_train_loader = loader(train_path_imgs, 
                            train_path_masks, 
                            SIZE, 
                            LENGTH, 
                            BATCH_SIZE_FOR_STAT, 
                            transform=None,
                            for_stat=True)


# instantiate the transform
train_transform = build_transform(stat_train_loader, mode='train')
val_transform = build_transform(stat_train_loader, mode='val')


TypeError: Can't instantiate abstract class GeoSampler with abstract method __iter__

In [86]:
print(next(iter(stat_train_loader))['mask'].shape)

torch.Size([16, 1, 256, 256])


In [95]:
from dataset.augmentations import compute_band_minmax
min, max = compute_band_minmax(stat_train_loader)

print(min)
print(max)


tensor([0., 0., 0., 0., 0., 0.])
tensor([2.6552e+08, 1.3891e+08, 2.3694e+08, 1.9537e+08, 2.8000e+08, 1.5849e+08])


In [35]:
train_transform

Compose([
  HorizontalFlip(p=0.5),
  VerticalFlip(p=0.5),
  RandomRotate90(p=0.5),
  OneOf([
    RandomBrightnessContrast(p=0.5, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True, ensure_safe_range=False),
    GaussNoise(p=0.5, std_range=(0.2, 0.44), mean_range=(0.0, 0.0), per_channel=True, noise_scale_factor=1.0),
    ColorJitter(p=0.5, brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.5, 0.5)),
  ], p=0.8),
  Normalize(p=1.0, mean=[6890000.0, 5150000.0, 3020000.0, 2910000.0, 1850000.0, 335000.0, 7300000.0, 4770000.0, 2585000.0, 2310000.0, 1560000.0, 315000.0, 7790000.0, 5460000.0, 3040000.0, 2720000.0, 2170000.0, 490000.0, 8900000.0, 6250000.0, 3700000.0, 3470000.0, 3160000.0, 630000.0, 8505000.0, 5530000.0, 3105000.0, 2695000.0, 1905000.0, 515000.0], std=[71005000.0, 68910000.0, 74840000.0, 69455000.0, 69490000.0, 26525000.0, 56010000.0, 56930000.0, 60815000.0, 62790000.0, 55040000.0, 29795000.0, 7070000.0, 8760000.0, 11460000.

In [ ]:
# instantiate the training dataloader

train_loader = loader(train_path_imgs, 
                            train_path_masks, 
                            SIZE, 
                            LENGTH, 
                            BATCH_SIZE_FOR_TRAINING, 
                            transform=train_transform,
                            for_stat=False)

val_loader = loader(val_path_imgs, 
                            val_path_masks, 
                            SIZE, 
                            LENGTH, 
                            BATCH_SIZE_FOR_TRAINING, 
                            transform=val_transform,
                            for_stat=False)

train_batch = next(iter(train_loader))
val_batch = next(iter(val_loader))

print("Train Image Shape:", train_batch['image'].shape)  # Expected: (batch_size, 6, H, W)
print("Val Image Shape:", val_batch['image'].shape)      # Expected: (batch_size, 6, H, W)

# Checking normalization values
print("Train Image Min/Max:", train_batch['image'].min(), train_batch['image'].max())
print("Val Image Min/Max:", val_batch['image'].min(), val_batch['image'].max())